In [32]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

In [33]:
df = pd.read_csv('Dataset .csv')
df.info(), df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         9551 non-null   int64  
 1   Restaurant Name       9551 non-null   object 
 2   Country Code          9551 non-null   int64  
 3   City                  9551 non-null   object 
 4   Address               9551 non-null   object 
 5   Locality              9551 non-null   object 
 6   Locality Verbose      9551 non-null   object 
 7   Longitude             9551 non-null   float64
 8   Latitude              9551 non-null   float64
 9   Cuisines              9542 non-null   object 
 10  Average Cost for two  9551 non-null   int64  
 11  Currency              9551 non-null   object 
 12  Has Table booking     9551 non-null   object 
 13  Has Online delivery   9551 non-null   object 
 14  Is delivering now     9551 non-null   object 
 15  Switch to order menu 

(None,
    Restaurant ID         Restaurant Name  Country Code              City  \
 0        6317637        Le Petit Souffle           162       Makati City   
 1        6304287        Izakaya Kikufuji           162       Makati City   
 2        6300002  Heat - Edsa Shangri-La           162  Mandaluyong City   
 3        6318506                    Ooma           162  Mandaluyong City   
 4        6314302             Sambo Kojin           162  Mandaluyong City   
 
                                              Address  \
 0  Third Floor, Century City Mall, Kalayaan Avenu...   
 1  Little Tokyo, 2277 Chino Roces Avenue, Legaspi...   
 2  Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...   
 3  Third Floor, Mega Fashion Hall, SM Megamall, O...   
 4  Third Floor, Mega Atrium, SM Megamall, Ortigas...   
 
                                      Locality  \
 0   Century City Mall, Poblacion, Makati City   
 1  Little Tokyo, Legaspi Village, Makati City   
 2  Edsa Shangri-La, Ortigas, Manda

In [34]:
df['Cuisines'] = df['Cuisines'].fillna('Other')

In [35]:
df['Cuisines'] = df['Cuisines'].apply(lambda x: x.split(',')[0].strip())

features = df[['Cuisines', 'Price range', 'Aggregate rating', 'Votes', 'Average Cost for two', 'City']].copy()

encoder = OneHotEncoder(sparse_output=False)
encoded_features = encoder.fit_transform(features[['Cuisines', 'Price range', 'City']])
encoded_features = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['Cuisines', 'Price range', 'City']))

In [36]:
scaler = MinMaxScaler()
rating_scaled = scaler.fit_transform(features[['Aggregate rating']])

In [37]:
combined_features = np.hstack([encoded_features, rating_scaled])

restaurant_names = df['Restaurant Name'].reset_index(drop=True)

combined_features.shape, restaurant_names.head()

((9551, 266),
 0          Le Petit Souffle
 1          Izakaya Kikufuji
 2    Heat - Edsa Shangri-La
 3                      Ooma
 4               Sambo Kojin
 Name: Restaurant Name, dtype: object)

In [38]:

def recommend_restaurants(cuisine_preference, price_range, city_preference, top_n=5):

    user_input = encoder.transform([[cuisine_preference, price_range, city_preference]])
    
    avg_rating = np.array([[0.5]])
    
    user_profile = np.hstack([user_input, avg_rating])
    
    similarity_scores = cosine_similarity(user_profile, combined_features)
    
    top_indices = similarity_scores[0].argsort()[-top_n:][::-1]

    recommendations = pd.DataFrame({
        'Restaurant Name': restaurant_names.iloc[top_indices].values,
        'Similarity Score': similarity_scores[0][top_indices]
    })
    
    return recommendations

In [39]:
recommend_restaurants('Chinese', 2, 'Bangalore')


c:\Users\bossg\miniconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Restaurant Name,Similarity Score
0,Froth On Top,0.69798
1,Chiyang,0.69798
2,Chimney Sizzlers,0.69798
3,Nanking,0.69798
4,Nagaland,0.69798


In [ ]:
# Interactive recommendation function for users
def interactive_recommendation():
    """
    Interactive function to get user preferences and provide recommendations
    """
    print("==== RESTAURANT RECOMMENDATION SYSTEM ====")
    print("Please enter your preferences to get personalized restaurant recommendations:")
    
    # FOOD PREFERENCES - Make this more prominent
    print("\n➡️ STEP 1: SELECT YOUR FOOD PREFERENCES")
    print("Available cuisines to choose from:")
    for i, cuisine in enumerate(top_cuisines, 1):
        print(f"{i}. {cuisine}", end="\t")
        if i % 4 == 0:
            print()
    
    cuisine_input = input("\n📝 Enter cuisine numbers (comma separated, e.g., 1,3,5): ")
    if cuisine_input.strip():
        cuisine_indices = [int(idx.strip()) - 1 for idx in cuisine_input.split(',')]
        selected_cuisines = [top_cuisines[i] for i in cuisine_indices if 0 <= i < len(top_cuisines)]
        print(f"✅ Selected cuisines: {', '.join(selected_cuisines)}")
    else:
        selected_cuisines = []
        print("⚠️ No cuisine selected. Recommendations will be based on other preferences.")
    
    # PRICE RANGE - Make this more prominent
    print("\n➡️ STEP 2: SELECT YOUR PRICE RANGE")
    print("1: $ (Budget friendly)")
    print("2: $$ (Moderately priced)")
    print("3: $$$ (High-end)")
    print("4: $$$$ (Luxury)")
    price_input = input("📝 Enter your preferred price range (1-4): ")
    price_range = int(price_input) if price_input.strip() and price_input.strip() in ["1", "2", "3", "4"] else None
    if price_range:
        print(f"✅ Selected price range: {price_range} {'$' * price_range}")
    else:
        print("⚠️ No price range selected. Recommendations will be based on other preferences.")
    
    # CITY SELECTION - Make this more prominent
    print("\n➡️ STEP 3: SELECT YOUR PREFERRED CITY")
    top_cities = processed_df['City'].value_counts().head(10)
    for i, (city, count) in enumerate(top_cities.items(), 1):
        print(f"{i}. {city} ({count} restaurants)")
    
    city_input = input("📝 Enter city number or leave blank for any city: ")
    selected_city = top_cities.index[int(city_input) - 1] if city_input.strip() and city_input.strip().isdigit() and 0 < int(city_input) <= len(top_cities) else None
    if selected_city:
        print(f"✅ Selected city: {selected_city}")
    else:
        print("⚠️ No city selected. Searching across all cities.")
    
    # ADDITIONAL PREFERENCES
    print("\n➡️ STEP 4: ANY ADDITIONAL PREFERENCES?")
    min_rating_input = input("📝 Enter minimum acceptable rating (0-5): ")
    min_rating = float(min_rating_input) if min_rating_input.strip() and min_rating_input.replace('.', '', 1).isdigit() else None
    if min_rating:
        print(f"✅ Minimum rating set to: {min_rating}/5")
    
    # Build preferences dictionary
    preferences = {}
    if selected_cuisines:
        preferences['cuisines'] = selected_cuisines
    if price_range:
        preferences['price_range'] = price_range
    if min_rating:
        preferences['min_rating'] = min_rating
    if selected_city:
        preferences['city'] = selected_city
    
    # Get number of recommendations
    n_input = input("\n📝 How many restaurants would you like to see? (default: 5): ")
    n_recommendations = int(n_input) if n_input.strip() and n_input.strip().isdigit() else 5
    
    # Check if we have any preferences at all
    if not preferences:
        print("⚠️ No preferences provided. Showing random popular restaurants.")
        # Default to highly rated restaurants if no preferences
        preferences['min_rating'] = 4.0
    
    # Get recommendations
    print("\n🔍 Generating recommendations based on your preferences...")
    print("Please wait...")
    recommendations = get_restaurant_recommendations(preferences, n_recommendations)
    
    # Display recommendations
    print("\n✨ YOUR TOP", n_recommendations, "RESTAURANT RECOMMENDATIONS ✨")
    for i, (_, row) in enumerate(recommendations.iterrows(), 1):
        print(f"\n{i}. 🍽️ {row['Restaurant Name']}")
        print(f"   Cuisines: {row['Cuisines']}")
        print(f"   Location: {row['City']}")
        print(f"   Rating: {'⭐' * int(row['Aggregate rating'])} ({row['Aggregate rating']}/5 - {row['Votes']} votes)")
        print(f"   Price: {row['Average Cost for two']} {row['Currency']} (Price range: {'$' * int(row['Price range'])})")
        print(f"   Match score: {row['similarity_score']:.2f}")
    
    # Evaluate recommendations
    print("\n📊 RECOMMENDATION QUALITY METRICS:")
    evaluate_recommendations(recommendations, preferences)
    
    # Ask if the user wants to make another search
    print("\nWould you like to search for more restaurants with different criteria?")
    another_search = input("Enter 'yes' to continue or any other key to exit: ")
    if another_search.lower() == 'yes':
        interactive_recommendation()
    else:
        print("Thank you for using our Restaurant Recommendation System! Enjoy your meal! 🍴")

# Run the interactive recommendation function
interactive_recommendation()